In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
from utils2 import one_hot_encode
import numpy as np

import datetime as dt
import h5py
import numpy as np
import os
import fnmatch
import time
import re
from sklearn.metrics import average_precision_score
from pandas import DataFrame
import pandas as pd

In [3]:
import sys
sys.path.append('/home/ychwang/jklib')
import jklib
from jklib.genome import *
from jklib import REFERENCE

In [4]:
h_chr  = loadBlatOutput(REFERENCE.RefFlat.hg38, by="chrom")

In [5]:
loc = locus('chr8:140300606-140300626-')
seq1=loc.twoBitFrag().upper()

위 코드가 실행되시면 됩니다

In [6]:
# default model
paths = ('models/spliceai{}.h5'.format(x) for x in range(1, 6))
models = [load_model(resource_filename('spliceai', x),compile=False) for x in paths]

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [ ]:
def predict_sequence_with_context2(dna_sequence,model,assembly='hg38'):
    encoded_dna_sequence = one_hot_encode(dna_sequence)[None, :]
    
    if strand == '-':
        encoded_dna_sequence = encoded_dna_sequence[:, ::-1, ::-1]
    
    predicted_value = np.mean([model[m].predict(encoded_dna_sequence) for m in range(5)], axis=0)
    
    
    acceptor_prob = predicted_value[0, :, 1]
    donor_prob = predicted_value[0, :, 2]
    
    acceptor_prob_sorted_index=acceptor_prob.argsort()
    acceptor_prob_sorted_index=np.flip(acceptor_prob_sorted_index)
    
    donor_prob_sorted_index=donor_prob.argsort()
    donor_prob_sorted_index=np.flip(donor_prob_sorted_index)
    
    acceptor_prob_sorted_value=acceptor_prob[acceptor_prob_sorted_index]
    donor_prob_sorted_value=donor_prob[donor_prob_sorted_index]
    
    #                 0          1             2                 3                          4
    return predicted_value, acceptor_prob, donor_prob, acceptor_prob_sorted_index, donor_prob_sorted_index,\
        acceptor_prob_sorted_value, donor_prob_sorted_value

In [ ]:
def replace_dna_ref_to_alt2(dna_sequence,pos,ref,alt,max_distance):    
    original_dna_sequence=dna_sequence
    altered_dna_sequence=dna_sequence[:5000+max_distance] + alt + dna_sequence[5000+max_distance+len(ref):]

    
    print("Ref and Alt : ("+ref+" to "+alt+")")
    if len(ref)>len(alt):
        print(original_dna_sequence[5000+max_distance-50:5000+max_distance]+ref+                         original_dna_sequence[5000+max_distance+len(ref):5000+max_distance+len(ref)+50])
        print(" "*50+"-"*len(ref))
        print(altered_dna_sequence[5000+max_distance-50:5000+max_distance]+alt+'.'*(len(ref)-len(alt))+altered_dna_sequence[5000+max_distance+len(alt):5000+max_distance+len(alt)+50])
        
        
    else:
        print(original_dna_sequence[5000+max_distance-50:5000+max_distance]+ref+'.'*(len(alt)-len(ref))+original_dna_sequence[5000+max_distance+len(ref):5000+max_distance+len(ref)+50])
        print(" "*50+"-"*len(alt))
        print(altered_dna_sequence[5000+max_distance-50:5000+max_distance]+alt                        +altered_dna_sequence[5000+max_distance+len(alt):5000+max_distance+len(alt)+50])
    print()
    
    print(str(len(original_dna_sequence))+' HEREHEREHR')
    print(str(len(altered_dna_sequence))+' HEREHEREHR')
    
    return original_dna_sequence,altered_dna_sequence

In [ ]:
def compare_mutated_sequences2(chrom,pos,ref,alt,strand,max_distance,model,assembly='hg38'):
    
    start_time = time.time()
    
    start_index= pos-5000-max_distance
    end_index= pos+5000+max_distance+1
    
    dna_sequence=jkgenome.locus(chrom+':'+str(start_index)+'-'+str(end_index)+'+')
    dna_sequence=dna_sequence.twoBitFrag2('hg38').upper()
    
    if dna_sequence[pos-start_index:pos-start_index+len(ref)]!=ref:
        print('Error in positioning')
        for j in range(-2,3):
            print(dna_sequence[5000+max_distance+j])
        return
    
    input_dna_sequence1,input_dna_sequence2=replace_dna_ref_to_alt2(dna_sequence,pos,ref,alt,max_distance)
    print ("sequence changed --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    predicted_value1,_,_,_,_,_,_ = predict_sequence_with_context2(input_dna_sequence1,model)
    predicted_value2,_,_,_,_,_,_ = predict_sequence_with_context2(input_dna_sequence2,model)
    print(input_dna_sequence1[:50])
    if strand == '-':
        predicted_value1 = predicted_value1[:, ::-1, ::1]
        predicted_value2 = predicted_value2[:, ::-1, ::1]
    
    print ("prediction --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
#     predicted_value_for_common1=np.concatenate((predicted_value1[0][:max_distance],predicted_value1[0][max_distance+len(ref):]))
#     predicted_value_for_common2=np.concatenate((predicted_value2[0][:max_distance],predicted_value2[0][max_distance+len(alt):]))
    predicted_value_for_common1=np.concatenate((predicted_value1[0][:max_distance],predicted_value1[0][-max_distance:]))
    predicted_value_for_common2=np.concatenate((predicted_value2[0][:max_distance],predicted_value2[0][-max_distance:]))
    
    
    
    predicted_value_for_raw1=predicted_value1[0][max_distance:max_distance+len(ref)]
    predicted_value_for_raw2=predicted_value2[0][max_distance:max_distance+len(alt)]
    
    acceptor_prob1 = predicted_value_for_common1[ :, 1]
    donor_prob1 = predicted_value_for_common1[ :, 2]
    
    acceptor_prob2 = predicted_value_for_common2[ :, 1]
    donor_prob2 = predicted_value_for_common2[ :, 2]
        
    
    delta_acceptor_prob = acceptor_prob1 - acceptor_prob2
    delta_donor_prob = donor_prob1 - donor_prob2
    
    delta_acceptor_sorted_index = (acceptor_prob1 - acceptor_prob2).argsort()
    delta_acceptor_sorted_index = np.flip(delta_acceptor_sorted_index)
    
    delta_donor_sorted_index = (donor_prob1 - donor_prob2).argsort()
    delta_donor_sorted_index = np.flip(delta_donor_sorted_index)
    
    delta_acceptor_sorted_value = delta_acceptor_prob[delta_acceptor_sorted_index]
    delta_donor_sorted_value = delta_donor_prob[delta_donor_sorted_index]

    
#     return predicted_value_for_common1,predicted_value_for_common2
    return delta_acceptor_prob, delta_donor_prob, delta_acceptor_sorted_index, delta_donor_sorted_index, \
            delta_acceptor_sorted_value, delta_donor_sorted_value, \
            predicted_value_for_raw1, predicted_value_for_raw2

In [ ]:
def view_result2(return_above,view_threshold,max_distance):
    (delta_acceptor_prob, delta_donor_prob, delta_acceptor_sorted_index, delta_donor_sorted_index, \
    delta_acceptor_sorted_value, delta_donor_sorted_value, predicted_value_for_raw1, predicted_value_for_raw2) = return_above
    
    count_acceptor_positive=0
    count_donor_positive=0
    for i in delta_acceptor_sorted_value:
        if i>=0:
            count_acceptor_positive+=1
    
    for i in delta_donor_sorted_value:
        if i>=0:
            count_donor_positive+=1
    
    delta_acceptor_sorted_index_copied=delta_acceptor_sorted_index.copy()
    delta_donor_sorted_index_copied=delta_donor_sorted_index.copy()
    
    for i in range(2*max_distance):
        if delta_acceptor_sorted_index_copied[i]>=max_distance:
            delta_acceptor_sorted_index_copied[i]+=1
        if delta_donor_sorted_index_copied[i]>=max_distance:
            delta_donor_sorted_index_copied[i]+=1
    
  
    df_acceptor_loss=pd.DataFrame()
    
    df_acceptor_loss['delta score(Acceptor Loss)']=delta_acceptor_sorted_value[:min(view_threshold,count_acceptor_positive)]
    df_acceptor_loss['pre-mRNA position']=(delta_acceptor_sorted_index_copied-max_distance)[:min(view_threshold,count_acceptor_positive)]
    
    
    
    
    delta_acceptor_sorted_value_r = delta_acceptor_sorted_value[::-1]
    delta_acceptor_sorted_index_r = delta_acceptor_sorted_index_copied[::-1]
    
    df_acceptor_gain=pd.DataFrame()
    
    df_acceptor_gain['delta score(Acceptor Gain)']=-delta_acceptor_sorted_value_r[:min(view_threshold,2*max_distance-count_acceptor_positive)]
    df_acceptor_gain['pre-mRNA position']=(delta_acceptor_sorted_index_r-max_distance)[:min(view_threshold,2*max_distance-count_acceptor_positive)]
    
    
    
    
    df_donor_loss=pd.DataFrame()
    
    df_donor_loss['delta score(Donor Loss)']=delta_donor_sorted_value[:min(view_threshold,count_donor_positive)]
    df_donor_loss['pre-mRNA position']=(delta_donor_sorted_index_copied-max_distance)[:min(view_threshold,count_donor_positive)]
    
    
    
    
    delta_donor_sorted_value_r = delta_donor_sorted_value[::-1]
    delta_donor_sorted_index_r = delta_donor_sorted_index_copied[::-1]
    
    df_donor_gain=pd.DataFrame()
    
    df_donor_gain['delta score(Donor Gain)']=-delta_donor_sorted_value_r[:min(view_threshold,2*max_distance-count_donor_positive)]
    df_donor_gain['pre-mRNA position']=(delta_donor_sorted_index_r-max_distance)[:min(view_threshold,2*max_distance-count_donor_positive)]
    
    
    
    ## START FOR RAW ##
    
    delta_in_raw = predicted_value_for_raw1-predicted_value_for_raw2
    delta_raw_acceptor= delta_in_raw[:, 1]
    delta_raw_donor= delta_in_raw[:, 2]
    
    
    count_delta_acceptor_positive=0
    count_delta_donor_positive=0
    for i in delta_raw_acceptor:
        if i>=0:
            count_delta_acceptor_positive+=1
    
    for i in delta_raw_donor:
        if i>=0:
            count_delta_donor_positive+=1
    

    
    delta_raw_acceptor_sorted_index_copied=delta_raw_acceptor.argsort().copy()
    delta_raw_donor_sorted_index_copied=delta_raw_donor.argsort().copy()
    
    delta_raw_acceptor_sorted_index_copied = np.flip(delta_raw_acceptor_sorted_index_copied)
    delta_raw_donor_sorted_index_copied = np.flip(delta_raw_donor_sorted_index_copied)
    
    delta_raw_acceptor_sorted_value=delta_raw_acceptor[delta_raw_acceptor_sorted_index_copied]
    delta_raw_donor_sorted_value=delta_raw_donor[delta_raw_donor_sorted_index_copied]
    
    
    
    for i in range(len(delta_raw_acceptor)):
        delta_raw_acceptor_sorted_index_copied[i]+=1
        delta_raw_donor_sorted_index_copied[i]+=1
    

    
  
    df_raw_acceptor_loss=pd.DataFrame()
    
    df_raw_acceptor_loss['delta score in raw(Acceptor Loss)']=delta_raw_acceptor_sorted_value[:min(view_threshold,count_delta_acceptor_positive)]
    df_raw_acceptor_loss['pre-mRNA position']=delta_raw_acceptor_sorted_index_copied[:min(view_threshold,count_delta_acceptor_positive)]
    
    
    
    
    delta_raw_acceptor_sorted_value_r = delta_raw_acceptor_sorted_value[::-1]
    delta_raw_acceptor_sorted_index_r = delta_raw_acceptor_sorted_index_copied[::-1]
    
    df_raw_acceptor_gain=pd.DataFrame()
    
    df_raw_acceptor_gain['delta score in raw(Acceptor Gain)']=-delta_raw_acceptor_sorted_value_r[:min(view_threshold,len(delta_raw_acceptor_sorted_value_r)-count_delta_acceptor_positive)]
    df_raw_acceptor_gain['pre-mRNA position']=delta_raw_acceptor_sorted_index_r[:min(view_threshold,len(delta_raw_acceptor_sorted_value_r)-count_delta_acceptor_positive)]
    
    
    
    df_raw_donor_loss=pd.DataFrame()
    
    df_raw_donor_loss['delta score in raw(Donor Loss)']=delta_raw_donor_sorted_value[:min(view_threshold,count_delta_donor_positive)]
    df_raw_donor_loss['pre-mRNA position']=delta_raw_donor_sorted_index_copied[:min(view_threshold,count_delta_donor_positive)]
    
    
    
    
    delta_raw_donor_sorted_value_r = delta_raw_donor_sorted_value[::-1]
    delta_raw_donor_sorted_index_r = delta_raw_donor_sorted_index_copied[::-1]
    
    df_raw_donor_gain=pd.DataFrame()
    
    df_raw_donor_gain['delta score in raw(Donor Gain)']=-delta_raw_donor_sorted_value_r[:min(view_threshold,len(delta_raw_donor_sorted_value_r)-count_delta_donor_positive)]
    df_raw_donor_gain['pre-mRNA position']=delta_raw_donor_sorted_index_r[:min(view_threshold,len(delta_raw_donor_sorted_value_r)-count_delta_donor_positive)]
    
    
    
    pd.options.display.float_format = '{:.5f}'.format
    
    return df_acceptor_loss,df_donor_loss,df_acceptor_gain,df_donor_gain,\
            df_raw_acceptor_loss,df_raw_donor_loss,df_raw_acceptor_gain,df_raw_donor_gain

In [7]:
chr='chr8'
pos=140300615
ref='T'
alt='G'
strand='-'
max_distance=50

x=compare_mutated_sequences2(chr,pos,ref,alt,strand,max_distance,models)

NameError: name 'compare_mutated_sequences2' is not defined

In [ ]:
view_result=view_result2(x,30,50)
view_result[0]

In [331]:
chr='chr1'
pos=930129
ref='C'
alt='G'
strand='+'
max_distance=50

x=compare_mutated_sequences2(chr,pos,ref,alt,strand,max_distance,models)

Ref and Alt : (C to G)
CAGCCCCACCTTCCTCTCCTCCTGCCCCACCTTCCTCTCCTCCTGCCCCACCTTCCTCTCCTCCTGCCCCACCAGAACCGGGGGCGGCTGGCAGACAAGAG
                                                  -
CAGCCCCACCTTCCTCTCCTCCTGCCCCACCTTCCTCTCCTCCTGCCCCAGCTTCCTCTCCTCCTGCCCCACCAGAACCGGGGGCGGCTGGCAGACAAGAG

sequence changed --- 0.002262592315673828 seconds ---
prediction --- 0.9056439399719238 seconds ---


In [319]:
chr='chr1'
pos=1042465
ref='GGGC'
alt='G'
strand='+'
max_distance=50

x=compare_mutated_sequences2(chr,pos,ref,alt,strand,max_distance,models)

Ref and Alt : (GGGC to G)
CCTCTGGCCTGTCCGCATGTCTCCACGCTGTAAGCACAGGCTGTTGGGGTGGGCAGGCTGGACACAACACCAGGGTCCTCACTCTTGGGACATGGGCAGCCGTC
                                                  ----
CCTCTGGCCTGTCCGCATGTCTCCACGCTGTAAGCACAGGCTGTTGGGGTG...AGGCTGGACACAACACCAGGGTCCTCACTCTTGGGACATGGGCAGCCGTC

sequence changed --- 0.0018095970153808594 seconds ---
prediction --- 1.039003610610962 seconds ---


In [341]:
chr='chr1'
pos=1042600
ref='A'
alt='AGGGTAGA'
strand='+'
max_distance=50

x=compare_mutated_sequences2(chr,pos,ref,alt,strand,max_distance,models)

Ref and Alt : (A to AGGGTAGA)
GGGCCAGGGGGAGGCAGTGGCCGTCGTGTCCCGTCGTGTTCGTCTTGGTCA.......GTTCGCCCATCTTCTGGTCTTTCTCTCTGCGTCTGTCTCTTCTCTTTGGT
                                                  --------
GGGCCAGGGGGAGGCAGTGGCCGTCGTGTCCCGTCGTGTTCGTCTTGGTCAGGGTAGAGTTCGCCCATCTTCTGGTCTTTCTCTCTGCGTCTGTCTCTTCTCTTTGGT

sequence changed --- 0.0020122528076171875 seconds ---
prediction --- 1.0425617694854736 seconds ---


In [441]:
chr='chr11'
pos=108299059
ref='G'
alt='GGCTCTACCCTTTGAAAGAAGGGCTCTCAAAAAATTTGTGAACATATTTTAGTAAATACCATCATGATGGAAAGAGCACGAAGAGCCCTCTGGGTTGCTGAAAATGTCCTGTCTTCATCTGGTTAGGGAGAACACAGGCAGAAAAAAATGTGTATTTGTATGTATAAATTTTCATTGAACTGTGTGTTTAAGATTATGGAATTTGTGTGATACCTCAACAAAAATTTTAACAGTTTTAAGGTTGATTCTCTCTCTCTTTTTTTTTTTTTCTTTTGTCACCCAGGCTAGAGTGCAGTGGTGCCATCTTGGCTCACTGCAGCCTCCGCCTCCTGGGTTCAAGTGATTCTCCTGCCTCAGCCTCCTGAGTAGCTGGGATTACGGGC'
strand='+'
max_distance=50

x=compare_mutated_sequences2(chr,pos,ref,alt,strand,max_distance,models)

Ref and Alt : (G to GGCTCTACCCTTTGAAAGAAGGGCTCTCAAAAAATTTGTGAACATATTTTAGTAAATACCATCATGATGGAAAGAGCACGAAGAGCCCTCTGGGTTGCTGAAAATGTCCTGTCTTCATCTGGTTAGGGAGAACACAGGCAGAAAAAAATGTGTATTTGTATGTATAAATTTTCATTGAACTGTGTGTTTAAGATTATGGAATTTGTGTGATACCTCAACAAAAATTTTAACAGTTTTAAGGTTGATTCTCTCTCTCTTTTTTTTTTTTTCTTTTGTCACCCAGGCTAGAGTGCAGTGGTGCCATCTTGGCTCACTGCAGCCTCCGCCTCCTGGGTTCAAGTGATTCTCCTGCCTCAGCCTCCTGAGTAGCTGGGATTACGGGC)
GTTAGACGTGAGTCACTAAGTACAGACCACACTCAACGGGAGGGAAGTTAG..............................................................................................................................................................................................................................................................................................................................................................................................GCTCTACCCTTTGAAAGAAGGGCTCTCAAAAAATTTGTGAACATATTTTA
                                                  -------------------------------------------------------------

In [442]:
# To visualzie

view_result=view_result2(x,50)

display(view_result[0])는 acceptor_loss,<br/>
display(view_result[1])는 donor_loss,<br/>
display(view_result[2])는 acceptor_gain,<br/>
display(view_result[3])는 donor_gain 를 출력해줍니다.<br/><br/>

display(view_result[4])는 ref와 alt내에서의 acceptor_loss,<br/>
display(view_result[5])는 ref와 alt내에서의 donor_loss,<br/>
display(view_result[6])는 ref와 alt내에서의 acceptor_gain,<br/>
display(view_result[7])는 ref와 alt내에서의 donor_gain 를 출력해줍니다.<br/><br/>

In [448]:
display(view_result[6])

,delta score in raw(Acceptor Gain),pre-mRNA position
0,0.16307,284
1,0.00382,289
2,0.00052,127
3,0.00031,53
4,0.00006,296
...,...,...
294,0.00000,32
295,0.00000,100
296,0.00000,348
297,0.00000,328


이 밑으로는 사용되지 않는 코드들입니다.

In [6]:
def predict_sequence_with_context(dna_sequence,strand,model,assembly='hg38'):
    encoded_dna_sequence = one_hot_encode(dna_sequence)[None, :]
    
    if strand == '-':
        encoded_dna_sequence = encoded_dna_sequence[:, ::-1, ::-1]
    
    predicted_value = np.mean([model[m].predict(encoded_dna_sequence) for m in range(5)], axis=0)
    
    
    acceptor_prob = predicted_value[0, :, 1]
    donor_prob = predicted_value[0, :, 2]
    
    acceptor_prob_sorted_index=acceptor_prob.argsort()
    acceptor_prob_sorted_index=np.flip(acceptor_prob_sorted_index)
    
    donor_prob_sorted_index=donor_prob.argsort()
    donor_prob_sorted_index=np.flip(donor_prob_sorted_index)
    
    acceptor_prob_sorted_value=acceptor_prob[acceptor_prob_sorted_index]
    donor_prob_sorted_value=donor_prob[donor_prob_sorted_index]
    
    #                 0          1             2                 3                          4
    return predicted_value, acceptor_prob, donor_prob, acceptor_prob_sorted_index, donor_prob_sorted_index,\
        acceptor_prob_sorted_value, donor_prob_sorted_value

In [7]:
def replace_dna_ref_to_alt(chrom,pos,ref,alt,start_index,end_index,strand,context,model,assembly='hg38'):
    dna_sequence=jkgenome.locus(chrom+':'+str(start_index)+'-'+str(end_index)+strand)
    dna_sequence=dna_sequence.twoBitFrag2('hg38').upper()
    
    if dna_sequence[pos-start_index:pos-start_index+len(ref)]!=ref:
        print('Error in positioning')
        
    original_dna_sequence=dna_sequence
    altered_dna_sequence=dna_sequence[:pos-start_index] + alt + dna_sequence[pos-start_index+len(ref):]
    
    print("Ref and Alt : ("+ref+" to "+alt+")")
    if len(ref)>len(alt):
        print(original_dna_sequence[pos-start_index-50:pos-start_index]+ref+                         original_dna_sequence[pos-start_index+len(ref):pos-start_index+len(ref)+50])
        print(" "*50+"-"*len(ref))
        print(altered_dna_sequence[pos-start_index-50:pos-start_index]+alt+'.'*(len(ref)-len(alt))+altered_dna_sequence[pos-start_index+len(alt):pos-start_index+len(alt)+50])
        
        
    else:
        print(original_dna_sequence[pos-start_index-50:pos-start_index]+ref+'.'*(len(alt)-len(ref))+original_dna_sequence[pos-start_index+len(ref):pos-start_index+len(ref)+50])
        print(" "*50+"-"*len(alt))
        print(altered_dna_sequence[pos-start_index-50:pos-start_index]+alt                        +altered_dna_sequence[pos-start_index+len(alt):pos-start_index+len(alt)+50])
    print()
    return original_dna_sequence,altered_dna_sequence

In [8]:
def compare_mutated_sequences(chrom,pos,ref,alt,start_index,end_index,strand,context,model,assembly='hg38'):
    start_time = time.time()
    
    dna_sequence=jkgenome.locus(chrom+':'+str(start_index)+'-'+str(end_index)+'+')
    dna_sequence=dna_sequence.twoBitFrag2('hg38').upper()
    
    if dna_sequence[pos-start_index:pos-start_index+len(ref)]!=ref:
        print('Error in positioning')
        return
    
    input_dna_sequence1,input_dna_sequence2=replace_dna_ref_to_alt2(chrom,pos,ref,alt,start_index,end_index,strand,context,model)
    print ("sequence changed --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    predicted_value1,_,_,_,_,_,_ = predict_sequence_with_context2(input_dna_sequence1,strand,model)
    predicted_value2,_,_,_,_,_,_ = predict_sequence_with_context2(input_dna_sequence2,strand,model)
    
    print ("prediction --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    predicted_value_for_common1=np.concatenate((predicted_value1[0][:pos-start_index],predicted_value1[0][pos-start_index+len(ref):]))
    predicted_value_for_common2=np.concatenate((predicted_value2[0][:pos-start_index],predicted_value2[0][pos-start_index+len(alt):]))
    
    predicted_value_for_raw1=predicted_value1[0][pos-start_index:pos-start_index+len(ref)]
    predicted_value_for_raw2=predicted_value2[0][pos-start_index:pos-start_index+len(alt)]
    
    acceptor_prob1 = predicted_value_for_common1[ :, 1]
    donor_prob1 = predicted_value_for_common1[ :, 2]
    
    acceptor_prob2 = predicted_value_for_common2[ :, 1]
    donor_prob2 = predicted_value_for_common2[ :, 2]
        
    
    delta_acceptor_prob = acceptor_prob1 - acceptor_prob2
    delta_donor_prob = donor_prob1 - donor_prob2
    
    delta_acceptor_sorted_index = (acceptor_prob1 - acceptor_prob2).argsort()
    delta_acceptor_sorted_index = np.flip(delta_acceptor_sorted_index)
    
    delta_donor_sorted_index = (donor_prob1 - donor_prob2).argsort()
    delta_donor_sorted_index = np.flip(delta_donor_sorted_index)
    
    delta_acceptor_sorted_value = delta_acceptor_prob[delta_acceptor_sorted_index]
    delta_donor_sorted_value = delta_donor_prob[delta_donor_sorted_index]
    
    
    
    return delta_acceptor_prob, delta_donor_prob, delta_acceptor_sorted_index, delta_donor_sorted_index, \
            delta_acceptor_sorted_value, delta_donor_sorted_value, \
            predicted_value_for_raw1, predicted_value_for_raw2

In [9]:
def view_delta(delta_acceptor_prob, delta_donor_prob, delta_acceptor_sorted_index, \
               delta_donor_sorted_index, delta_acceptor_sorted_value, delta_donor_sorted_value, \
            predicted_value_for_raw1, predicted_value_for_raw2,ranking):
    print("Largest Delta in acceptor")
    for i in range(ranking):
        print('%10d : '% delta_acceptor_sorted_index[i],delta_acceptor_sorted_value[i] )
    print()
    
    print("Smallest Delta in acceptor")
    for i in range(-1,-ranking-1,-1):
        print('%10d : '% delta_acceptor_sorted_index[i],delta_acceptor_sorted_value[i] )
    print()
    
    print("Largest Delta in donor")
    for i in range(ranking):
        print('%10d : '% delta_donor_sorted_index[i],delta_donor_sorted_value[i] )
    print()
    
    print("Smallest Delta in donor")
    for i in range(-1,-ranking-1,-1):
        print('%10d : '% delta_donor_sorted_index[i],delta_donor_sorted_value[i] )
    print()
    print("Prob for Ref("+ref+')')
    print(predicted_value_for_raw1)
    print()
    print("Prob for Alt("+alt+')')
    print(predicted_value_for_raw2)
    print()

In [42]:
pos=108299060
ref='T'
alt='G'

x=compare_mutated_sequences('chr8',140300615,ref,alt,140295565,140305666,'+',10000,models)

Ref and Alt : (T to G)
CATTTTGTGTGGCCGGAGGCTAGCAGGAAGGTTCAATAGTTTCACATGCCTGTTGTTTCAAACAGAACACAAATACTTCAACTGTCTGGTTAGCGTGGTTT
                                                  -
CATTTTGTGTGGCCGGAGGCTAGCAGGAAGGTTCAATAGTTTCACATGCCGGTTGTTTCAAACAGAACACAAATACTTCAACTGTCTGGTTAGCGTGGTTT

sequence changed --- 0.0021009445190429688 seconds ---
prediction --- 4.471197128295898 seconds ---


In [340]:
x[-1],x[-2]

(array([[9.9998838e-01, 5.9908612e-06, 5.6180970e-06],
        [9.9997568e-01, 5.3283616e-06, 1.8994824e-05],
        [8.8686007e-01, 1.7099734e-02, 9.6040204e-02],
        [9.9997902e-01, 1.6930042e-05, 4.0703262e-06],
        [9.9997061e-01, 1.7508986e-05, 1.1816104e-05],
        [9.9995959e-01, 2.5733127e-05, 1.4624675e-05],
        [9.9996978e-01, 2.6363025e-05, 3.8955200e-06],
        [9.9624860e-01, 3.7170320e-03, 3.4272922e-05],
        [8.2308722e-01, 4.2800061e-04, 1.7648479e-01]], dtype=float32),
 array([[9.993326e-01, 8.073356e-05, 5.866751e-04]], dtype=float32))

In [300]:
encoded_dna_sequence1 = one_hot_encode('N'*(10000//2) + ctestInput1 + 'N'*(10000//2))[None, :]
encoded_dna_sequence2 = one_hot_encode('N'*(10000//2) + ctestInput2 + 'N'*(10000//2))[None, :]

In [314]:
ctestInput1=""
for i in range(len(testInput1)):
    ctestInput1+=testInput1[-i]

In [315]:
ctestInput2=""
for i in range(len(testInput2)):
    ctestInput2+=testInput2[-i]

In [317]:
ctestInput1[:30],testInput1[-30:]

('GAGGTTTGCGGGAGTAAACAAGAAGTAGAG', 'TGAGATGAAGAACAAATGAGGGCGTTTGGA')

In [313]:
for i in range(200000):
    if ctestInput1[i]!=ctestInput2[i]:
        print("diff at "+str(i)+" "+ctestInput1[i],ctestInput2[i])

diff at 99999 A C


In [299]:
len(ctestInput1),len(ctestInput2)

(200000, 200000)

In [281]:
def predict_sequence_with_context222(dna_sequence,context,model,assembly='hg38'):
    encoded_dna_sequence = one_hot_encode('N'*(context//2) + dna_sequence + 'N'*(context//2))[None, :]
    for i in range(len(encoded_dna_sequence)):
        if encoded_dna_sequence[0][i]!=0:
            encoded_dna_sequence[0][i]=5-encoded_dna_sequence[0][i]
            
    
    predicted_value = np.mean([model[m].predict(encoded_dna_sequence) for m in range(5)], axis=0)
    
    
    acceptor_prob = predicted_value[0, :, 1]
    donor_prob = predicted_value[0, :, 2]
    
    acceptor_prob_sorted_index=acceptor_prob.argsort()
    acceptor_prob_sorted_index=np.flip(acceptor_prob_sorted_index)
    
    donor_prob_sorted_index=donor_prob.argsort()
    donor_prob_sorted_index=np.flip(donor_prob_sorted_index)
    
    acceptor_prob_sorted_value=acceptor_prob[acceptor_prob_sorted_index]
    donor_prob_sorted_value=donor_prob[donor_prob_sorted_index]
    
    #                 0          1             2                 3                          4
    return predicted_value, acceptor_prob, donor_prob, acceptor_prob_sorted_index, donor_prob_sorted_index,\
        acceptor_prob_sorted_value, donor_prob_sorted_value

In [450]:
pp=jkgenome.locus('chr8:140300565-140300666+')
testInput1=pp.twoBitFrag2('hg38').upper()
len(testInput1)

101

In [451]:
testInput2=testInput1[:50-1]+'G'+testInput1[50:]

In [453]:
len(testInput1),len(testInput2),testInput2[50]

(101, 101, 'T')

In [479]:
testResult1=predict_sequence_with_context(testInput1,10000,models,assembly='hg38')
testResult2=predict_sequence_with_context(testInput2,10000,models,assembly='hg38')

In [489]:
len(testResult1[0][0])

101

In [490]:
testResult1[5][:30]

array([3.23750041e-02, 8.04164819e-03, 3.92570253e-03, 1.12120085e-03,
       8.09352379e-04, 4.49840212e-04, 3.18651873e-04, 2.35539075e-04,
       1.75542213e-04, 1.65320002e-04, 1.60658747e-04, 1.57047558e-04,
       1.32356552e-04, 1.31460110e-04, 1.26880783e-04, 1.22217272e-04,
       1.21049445e-04, 1.20567624e-04, 1.14949376e-04, 1.10036184e-04,
       1.06633015e-04, 1.05626845e-04, 9.79065298e-05, 8.48221025e-05,
       8.17297914e-05, 6.53561874e-05, 5.77614483e-05, 5.64154980e-05,
       5.33474304e-05, 4.97122455e-05], dtype=float32)

In [491]:
testResult1[3][:30]

array([65, 87, 93, 82, 70, 72, 84, 39, 61, 18, 88, 96, 95, 76, 86, 26, 23,
       29, 77, 91, 81, 74, 47, 45, 85, 69, 94, 90, 83, 97])

In [492]:
testResult2[5][:30]

array([3.6704972e-02, 7.6620854e-03, 3.9339373e-03, 1.2013034e-03,
       9.3438383e-04, 4.4399663e-04, 3.1739305e-04, 2.4963630e-04,
       1.8139134e-04, 1.7632445e-04, 1.6302233e-04, 1.5322703e-04,
       1.4985586e-04, 1.4926252e-04, 1.4824625e-04, 1.3275603e-04,
       1.2798907e-04, 1.1267855e-04, 1.0360809e-04, 9.6483695e-05,
       9.5840696e-05, 9.3472925e-05, 9.1332331e-05, 9.1121561e-05,
       9.0461377e-05, 6.5233624e-05, 6.2680447e-05, 5.6511355e-05,
       5.5614586e-05, 5.2507639e-05], dtype=float32)

In [493]:
testResult2[3][:30]

array([65, 87, 93, 82, 70, 72, 84, 39, 18, 61, 26, 76, 95, 88, 96, 86, 23,
       45, 81, 77, 74, 29, 91, 85, 47, 69, 94, 90, 30, 97])

In [495]:
xx=(testResult1[1]-testResult2[1])
xx.sort()
print(xx[:10])
print(xx[-10:])
pp=xx.argsort()

print(pp[10:])
print(pp[-10:])

[-4.3299682e-03 -1.2503145e-04 -8.0102589e-05 -4.0805055e-05
 -2.7856448e-05 -2.1766915e-05 -1.7499304e-05 -1.6357160e-05
 -1.6071339e-05 -1.4097226e-05]
[6.6014127e-06 7.4451527e-06 8.8013039e-06 9.7861484e-06 1.1396231e-05
 1.4200458e-05 1.8465682e-05 1.8703853e-05 2.7094698e-05 3.7956284e-04]
[ 10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63
  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99
 100]
[ 91  92  93  94  95  96  97  98  99 100]


In [485]:
xx=(testResult1[1]-testResult2[1])

In [486]:
np.array(range(len(testResult1[1])))[xx>0.9]

array([], dtype=int64)

In [478]:
len(testResult1[1])

20000

In [430]:
testResult1[6][:30]-testResult2[6][:30]

array([ 0.0000000e+00, -1.7285347e-06,  0.0000000e+00,  3.1054020e-04,
        0.0000000e+00,  3.2675266e-04,  9.2566013e-05,  0.0000000e+00,
        1.1127591e-03,  5.6468666e-02,  5.7368875e-02,  9.5674396e-03,
        3.0140281e-02,  1.6888976e-03,  0.0000000e+00,  3.7291646e-04,
        1.3731724e-01,  6.0564443e-02,  2.9785335e-03,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  5.2675605e-05,  5.7145953e-05,
        0.0000000e+00,  3.4645200e-07,  3.2436103e-04,  0.0000000e+00,
        3.5911426e-04, -7.5788125e-03], dtype=float32)

In [441]:
xx=(testResult1[5]-testResult2[5])

In [442]:
xx.sort()

In [443]:
xx[:10],xx[-10:]

(array([-0.10696635, -0.07015467, -0.05700406, -0.0321241 , -0.03099149,
        -0.02593975, -0.01960662, -0.01747973, -0.01663751, -0.01628403],
       dtype=float32),
 array([1.2141466e-04, 4.0273368e-04, 8.4060431e-04, 4.2890906e-03,
        4.5473576e-03, 5.5983633e-02, 6.6358447e-02, 7.4043334e-02,
        1.4059460e-01, 3.0369344e-01], dtype=float32))

In [236]:
maxx-minn

0.8521849

In [246]:
len(testInput1)

200000

In [262]:
len(testResult1[0][0])

200000

In [ ]:
len(testInput)
testInput[1000000-10:1000000+10]

In [ ]:
pp=jkgenome.locus('chr8:139300616-141300616+')
testInput=pp.twoBitFrag2('hg38').upper()
testInput[1000000-1]

In [ ]:
testResult1[0][0][1000000-10:1000000+10]

In [ ]:
testResult=predict_sequence_with_context(testInput,10000,models,assembly='hg38')

In [ ]:
testResult[5][:30]

In [ ]:
testResult[3][:30] 

In [ ]:
testResult[3][:30]

In [ ]:
testResult1[0][0][69143]

In [ ]:
testInput2=testInput[:100000]+'G'+testInput[100000+1:]

In [ ]:
testResult2=predict_sequence_with_context(testInput2,10000,models,assembly='hg38')

In [ ]:
testResult2[3][:30]

In [404]:
pp=jkgenome.locus('chr11:47359708-47369710-')
xxxx=pp.twoBitFrag2('hg19').upper()
xxxx

'CTGTCTGAATGTTTGGGCGATGGGTGAAGGTGAGTTGAGAGATGGGTGGGAATTGGGGACAATAAACCGTCCCACCTGCCTGGTACCTTGACCCTGGGAAAGGCTGGGAAGGTGAGATCCTGGGGCAGATGCCCAGCCTCATGGGGTCATGAATGGGCAAGTCTGTGAATACTCAGAGGCCGCCCCCAGGGCAGGGCTTCTCAAACGGCCCCCTCTGAAGCCCCTTCCCCCATCTCTCCACCCTTTGAACCCAGAGGCCATGGGCACCGGAGACCTGGACCTCCTATCAGCCTTCCGCCGCACGTGAGTGGCCATCCTCAGGGCCTGGGGGAGGCCAGTGCTGGAGTCTGAGGCCCTTCAGGGTCTCGACTGGGGGTCAGGGCTGGGGATGATTTGCGGGGCGGAGCTGAAGTCAGTGGGGGCTGCAGGGGAGAGCAAGCTTCCCAGGCAGGTGAGGATACTGAGTCTAACCCCCACAGGAGCCTGGCTGGAGGTGGTCGGCGGATCAGGTACCCCTGCCCCAGGCCCTACCTGCACCATCCGCAGACCCCCAGGGGTCTGAGACGGGTGGGAGCCCAATCTGGCTAGTGTCCCTTTCTCCCTCCCCACTCCACTCCCATCCTGCTCCTAATCCCTTTCCAGTCCCTCACTGCAGCCTCACTGGGGGTCCCTCTCCATAGTGATAGCCATGAGGACACTGGGATTCTGGACTTCAGCTCACTGCTGAAAAAGAGGTGAGTCCTGGGTGGCAGTTCCTGGGGCAGCTGGTGAGGCCTGGCTCCAAATCCCAGCACTGTGGTTTGCCAGCTGTGTGACCCTGAGCACAACACTGCACCTCTCTGAGTTGGTTTCCTTGTCTGTCGAGGGGATGATCCCAGTGTGGAAGGAGTTAAAGGGCTCTGCAGACATTATGTCCCGTCAACAGTCATCCTCACAGTGTCCCCCACGGCCACTCCACACTTGAGGCTGCAAAAGCTCCCTCCGTAAACTCCGGGGACC

In [407]:
pppppp=predict_sequence_with_context(xxxx,10000,models,assembly='hg19')

In [409]:
pppppp[5][:30]

array([0.9986476 , 0.9976985 , 0.99746644, 0.9963444 , 0.9960413 ,
       0.9932972 , 0.98099554, 0.98063487, 0.9355999 , 0.8594283 ,
       0.74491316, 0.6864151 , 0.64838743, 0.6317855 , 0.5831027 ,
       0.54514587, 0.46745053, 0.4419094 , 0.44165546, 0.4244079 ,
       0.40271544, 0.361901  , 0.35564238, 0.34413013, 0.32823086,
       0.2702531 , 0.256715  , 0.2411994 , 0.17268628, 0.16499874],
      dtype=float32)

In [410]:
pppppp[3][:30]

array([6915, 8369, 9480, 6003, 5415, 5014, 5224, 7127, 8755, 4535, 1515,
        680, 1770,  642, 1789,  254, 4897, 1130, 4543, 3963, 6008, 9376,
        479, 4567, 1518, 4774, 1728, 4546, 1582, 4981])

In [358]:
input2=xxxx[:9999]+'T'+xxxx[10000:]

In [359]:
for i in range(20000):
    if input1[i]!=input2[i]:
        print(i,input1[i],input2[i])

9999 A T


In [360]:
len(input2),len(input1)

(20000, 20000)

In [ ]:
compare_mutated_sequences(chrom,pos,ref,alt,start_index,end_index,strand,context,model,assembly='hg38')

In [498]:
ref='T'
alt='G'
x=compare_mutated_sequences('chr8',140300616-1,ref,alt,140300616-5000,140300616+5000,'+',10000,models)

Ref and Alt : (T to G)
CATTTTGTGTGGCCGGAGGCTAGCAGGAAGGTTCAATAGTTTCACATGCCTGTTGTTTCAAACAGAACACAAATACTTCAACTGTCTGGTTAGCGTGGTTT
                                                  -
CATTTTGTGTGGCCGGAGGCTAGCAGGAAGGTTCAATAGTTTCACATGCCGGTTGTTTCAAACAGAACACAAATACTTCAACTGTCTGGTTAGCGTGGTTT

sequence changed --- 0.007157564163208008 seconds ---
prediction --- 3.5381619930267334 seconds ---


In [499]:
len(x[0])

9999

In [500]:
minn=1
maxx=0
for i in range(len(x[0])):
    if minn>x[0][i]:
        minn=x[0][i]
    if maxx<x[0][i]:
        maxx=x[0][i]
print(minn,maxx)

-0.00033577578 0.0008279085


In [382]:
######### TODAY


ref='A'
alt='G'

x=compare_mutated_sequences('chr3',142740189,ref,alt,142740190-10000,142740190+10000,'+',10000,models)

Ref and Alt : (A to G)
GGGACTACTGTATTATAAAAACTAATAATTGTCTTCTTTTCTCCCCTTTAAGTGGACGCGCCTTCAAGAAGAAATAGATCATCTGGTGGTAATACAGTTTT
                                                  -
GGGACTACTGTATTATAAAAACTAATAATTGTCTTCTTTTCTCCCCTTTAGGTGGACGCGCCTTCAAGAAGAAATAGATCATCTGGTGGTAATACAGTTTT

sequence changed --- 0.02477574348449707 seconds ---
prediction --- 2.76558780670166 seconds ---


In [383]:
view_delta(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],10)

Largest Delta in acceptor
     10000 :  0.9628092
     10093 :  0.13181767
      9901 :  0.009439922
     10025 :  0.009007661
     10019 :  0.005023396
     10016 :  0.0038957505
     11503 :  0.0014853477
     10123 :  0.0011556149
      7320 :  0.0008781776
      9697 :  0.00045300456

Smallest Delta in acceptor
      9999 :  -0.99900365
      9228 :  -0.0013905545
     13260 :  -0.000699684
     12625 :  -0.00038695335
      9258 :  -0.00036087027
      9250 :  -0.000225842
     13134 :  -0.00020573661
     10141 :  -0.0001413891
     13323 :  -0.00010251999
     12595 :  -7.840991e-05

Largest Delta in donor
      9935 :  0.008949852
     11714 :  0.004448712
      8328 :  0.0027533919
      8314 :  0.0011265688
      9941 :  0.0009659065
     12668 :  0.00061035156
      8687 :  0.0004094243
     10205 :  0.00027507544
     11312 :  0.00010192394
     10154 :  7.92357e-05

Smallest Delta in donor
     10032 :  -0.39879802
     10035 :  -0.09092188
      9329 :  -0.0068995804
    

In [386]:
######### TODAY


ref='G'
alt='A'

x=compare_mutated_sequences('chr11',47364709,ref,alt,47364709-10000,47364709+10000,'-',10000,models)

Ref and Alt : (G to A)
GGCGCACTGGGCCCGGAGGGGTGTCCGCAGCTTTCCTGCCACTTCCCTGCGGCCCCCACCCAGGTACATCTTTGAGTCCATCGGTGCCAAGCGTACCCTGA
                                                  -
GGCGCACTGGGCCCGGAGGGGTGTCCGCAGCTTTCCTGCCACTTCCCTGCAGCCCCCACCCAGGTACATCTTTGAGTCCATCGGTGCCAAGCGTACCCTGA

sequence changed --- 0.04125475883483887 seconds ---
prediction --- 2.791537284851074 seconds ---


In [387]:
view_delta(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],10)

Largest Delta in acceptor
     10012 :  0.40365905
      9896 :  0.013271779
     10057 :  0.0032427367
      9566 :  0.0031074584
      9900 :  0.0026617134
      9542 :  0.0026085973
     11006 :  0.0018072128
     10040 :  0.001747489
     10706 :  0.0010415763
      9852 :  0.0009825248

Smallest Delta in acceptor
     10001 :  -0.9829251
      9980 :  -0.11118871
      9876 :  -0.005281953
      9773 :  -0.002778083
      9878 :  -0.0022020414
      9873 :  -0.0020470626
      9871 :  -0.0018259231
      6788 :  -0.0018077493
      6517 :  -0.0016589463
      6511 :  -0.0016127378

Largest Delta in donor
     10096 :  0.005030662
     10089 :  0.004070699
      9666 :  0.0020329356
     10788 :  0.0014981255
      9565 :  0.0007424839
      6877 :  0.0005048588
     10327 :  0.00040739775
     11821 :  0.00016477704
     10579 :  0.00014519691
     10100 :  0.00011189486

Smallest Delta in donor
      9898 :  -0.003786683
     10011 :  -0.0033223256
      9046 :  -0.0023861006
   

In [ ]:


input_sequence = 'CGATCTGACGTGGGTGTCATCGCATTATCGATATTGCAT'
# Replace this with your custom sequence

context = 10000
paths = ('models/spliceai{}.h5'.format(x) for x in range(1, 6))
models = [load_model(resource_filename('spliceai', x),compile=False) for x in paths]
x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
y = np.mean([models[m].predict(x) for m in range(5)], axis=0)



In [ ]:
def replace_dna(chrom,pos,ref,alt,start_index,end_index,strand,assembly='hg38'):
    locus_region=jkgenome.locus(chrom+':'+str(start_index)+'-'+str(end_index)+strand)
    
    original_seq=locus_region.twoBitFrag2(assembly).upper()
    
    if original_seq[pos-start_index:pos-start_index+len(ref)]!=ref:
        print('Error in positioning')
        return '',''

    gene_seq1=original_seq
    gene_seq2=original_seq[:pos-start_index] + alt + original_seq[pos-start_index+len(ref):]
    
    print("original seq and changed seq : ("+ref+" to "+alt+")")
    if len(ref)>len(alt):
        print(gene_seq1[pos-start_index-50:pos-start_index]+ref+                         gene_seq1[pos-start_index+len(ref):pos-start_index+len(ref)+50])
        print(" "*50+"-"*len(ref))
        print(gene_seq2[pos-start_index-50:pos-start_index]+alt+'.'*(len(ref)-len(alt))+gene_seq2[pos-start_index+len(alt):pos-start_index+len(alt)+50])
        
        
    else:
        print(gene_seq1[pos-start_index-50:pos-start_index]+ref+'.'*(len(alt)-len(ref))+gene_seq1[pos-start_index+len(ref):pos-start_index+len(ref)+50])
        print(" "*50+"-"*len(alt))
        print(gene_seq2[pos-start_index-50:pos-start_index]+alt                        +gene_seq2[pos-start_index+len(alt):pos-start_index+len(alt)+50])
    print()
    return gene_seq1,gene_seq2

In [ ]:
def spliceAI_run(chrom,pos,ref,alt,start_index,end_index,strand,assembly='hg38'):  # hg38, pos-1; indel exam: T to TA
    start_time = time.time()
    
    input_seq1,input_seq2=replace_dna(chrom,pos,ref,alt,start_index,end_index,strand,assembly='hg38')
    
   
    
    
    
    x1 = one_hot_encode('N'*(context//2) + input_seq1 + 'N'*(context//2))[None, :]
    x2 = one_hot_encode('N'*(context//2) + input_seq2 + 'N'*(context//2))[None, :]
    print ("encoding --- %s seconds ---" % (time.time() - start_time))
    print()
    y1 = np.mean([models[m].predict(x1) for m in range(5)], axis=0)
    y2 = np.mean([models[m].predict(x2) for m in range(5)], axis=0)
    
    yy1=np.concatenate((y1[0][:pos-start_index],y1[0][pos-start_index+len(ref):]))
    yy2=np.concatenate((y2[0][:pos-start_index],y2[0][pos-start_index+len(alt):]))
    
    acceptor_prob_origianl = yy1[ :, 1]
    acceptor_prob_changed = yy2[ :, 1]
    donor_prob_original = yy1[ :, 2]
    donor_prob_changed = yy2[ :, 2]
    
    delta_acceptor=acceptor_prob_changed-acceptor_prob_origianl
    delta_donor=donor_prob_changed-donor_prob_original
    
    acceptor_watching=delta_acceptor.argsort()
    donor_watching=delta_donor.argsort()
    for i in range(-1,-10,-1):
        print(delta_acceptor[acceptor_watching[i]])
        
    for i in range(-1,-10,-1):
        print(delta_donor[donor_watching[i]])
    
    print ("predict --- %s seconds ---" % (time.time() - start_time))
    return (delta_acceptor,acceptor_watching,delta_donor,donor_watching)

In [ ]:
pos=140200046
ref='CTTTACA'
alt='AAAAAAAAAAAAAA'

predicted=spliceAI_run('chr8',pos,ref,alt,140200046-100000,140200046+100000,'+')

In [ ]:
predicted[0][predicted[1][-10:]],predicted[1][-10:]

In [ ]:
len(predicted[0])